In [92]:
import pandas as pd
import swifter
import re

In [2]:
raw = pd.read_csv("mails_raw.csv", index_col=0)

# Employee scrape

### Get employee names from email sender that contain "@enron"

In [19]:
df = pd.DataFrame()
df["Addrs"] = list(filter(lambda x: "@enron" in x, raw["From"].unique()))
df["From"] = df["Addrs"].swifter.apply(lambda x: raw[raw["From"]==x]["X-From"].unique())

### Clean-up names

In [26]:
df["Name"] = [0 for i in range(len(df))]

In [50]:
for idx in range(len(df)): df.loc[idx, "Name"] = df.loc[idx, "From"][0]

In [63]:
df["Name"] = df["Name"].apply(lambda x: x.split(" <")[0] if "<" in x else x)

In [72]:
df["Name"] = df["Name"].apply(lambda x: (" ".join(x.split(", ")[::-1])).replace(".","") if "," in x else x)

### Manual outliers fixed (total 5)

In [108]:
df[df["Name"].apply(lambda x: "Don" in x)]

,Addrs,From,Name
8,don.baughman@enron.com,"[Baughman Jr., Don </O=ENRON/OU=NA/CN=RECIPIEN...",Don Baughman Jr
37,tom.donohoe@enron.com,"[Donohoe, Tom </O=ENRON/OU=NA/CN=RECIPIENTS/CN...",Tom Donohoe
47,lindy.donoho@enron.com,"[Donoho, Lindy </O=ENRON/OU=NA/CN=RECIPIENTS/C...",Lindy Donoho
169,40enron@enron.com,[Don Black/HOU/EES@EES@ENRON <IMCEANOTES-Don+2...,Don Black/HOU/EES@EES@ENRON


In [109]:
df.loc[169, "Name"] = "Don Black"

### Read through print for anomalies

In [91]:
for name in df.sort_values("Name")["Name"]:
    print(name)

Albert Meyers
Amelia Alder
Andrea Ring
Andrew H Lewis
Andrew H Lewis
Andy Zipper
Angela McCulloch
Barry Tycholiz
Benjamin Rogers
Beverly Stephens
Bill Rapp
Bill Williams III
Brad Mckay
Brenda Whitehead
Brian Hoskins
Cara Semperger
Carl Tricoli
Carol Coats
Carol Kincannon
Carol St Clair
Casey Evans
Cathy Phillips
Cheryl Johnson
Chris Dorland
Chris Germany
Cindy Stark
Clint Dean
Cooper Richey
Craig Dean
Dan J Hyvl
Dana Davis
Danny McCarty
Daren J Farmer
Daren J Farmer
Darin Talley
Darrell Schoolcraft
Darron C Giron
Darron C Giron
David Forster
David W Delainey
David W Delainey
Debra Bailey
Debra Perlingiere
Diana Scholtes
Don Baughman Jr
Don Black/HOU/EES@EES@ENRON
Doug Gilbert-smith
Drew Fossum
Dutch Quigley
Elena Chilkina
Elizabeth Sager
Eric Bass
Eric Linder
Eric Moon
Eric Saibi
Errol McLaughlin Jr
Fletcher J Sturm
Fletcher J Sturm
Frank Ermis
Gaby Adamek
Geir Solberg
Geoff Storey
Gerald Nemec
Greg Couch
Greg Whalley
Gretel Smith
Harry Arora
Holden Salisbury
Hunter S Shively
Hunter S 

### Build Employee DataFrame

In [135]:
employees = pd.DataFrame()
employees["Name"] = df["Name"].unique()
employees["Last"] = employees["Name"].apply(lambda x: x.split(" ")[-1])
employees["First"] = employees["Name"].apply(lambda x: x.split(" ")[0])
employees.sort_values("Last", inplace=True)
employees.reset_index(inplace=True, drop=True)

### Used mail addresses

In [141]:
employees["Mail"] = employees["Name"].apply(lambda x: list(df[df["Name"]==x]["Addrs"]))

### Save

In [146]:
employees.drop(columns="Name").to_csv("employees.csv")

# Networking

In [151]:
employees[employees["First"] == "Kenneth"]["Mail"]

,Name,Last,First,Mail
107,Kenneth Lay,Lay,Kenneth,[kenneth.lay@enron.com]


In [207]:
def get_addrs(name="", first="", last=""):
    if name != "": return employees[employees["Name"] == name]["Mail"].iloc[0]
    if first != "": return employees[employees["First"] == first]["Mail"].iloc[0]
    if last != "": return employees[employees["Last"] == last]["Mail"].iloc[0]
    
idx_from_addrs = lambda x: employees["Mail"].apply(lambda y: x in y).argmax()

In [286]:
def sent_by(email):
    return pd.concat([raw[raw["From"] == m] for m in email])
sent_by(get_addrs(last="Davis"))

,Content,Message-ID,Date,From,To,Subject,Cc,Mime-Version,Content-Type,Content-Transfer-Encoding,Bcc,X-From,X-To,X-cc,X-bcc,X-Folder,X-Origin,X-FileName
20201,\n\n -----Original Message-----\nFrom: \tDavis...,<17542799.1075851584084.JavaMail.evans@thyme>,2001-09-14 10:24:26-07:00,dana.davis@enron.com,grace.taylor@enron.com,FW: Review,NaN,1.0,text/plain; charset=us-ascii,7bit,NaN,"Davis, Dana </O=ENRON/OU=NA/CN=RECIPIENTS/CN=D...","Taylor, Grace </O=ENRON/OU=NA/CN=RECIPIENTS/CN...",NaN,NaN,\DDAVIS (Non-Privileged)\Sent Items,DAVIS-D,DDAVIS (Non-Privileged).pst
20202,"Yes, I have thought about being an Admin. I j...",<11999742.1075851584061.JavaMail.evans@thyme>,2001-09-26 06:25:17-07:00,dana.davis@enron.com,lesley.ayers@enron.com,RE: JUS WANNA KNOW,NaN,1.0,text/plain; charset=us-ascii,7bit,NaN,"Davis, Dana </O=ENRON/OU=NA/CN=RECIPIENTS/CN=D...","Ayers, Lesley </O=ENRON/OU=NA/CN=RECIPIENTS/CN...",NaN,NaN,\DDAVIS (Non-Privileged)\Sent Items,DAVIS-D,DDAVIS (Non-Privileged).pst
20203,"Yes Bobbie, I will confirm that but hopefully ...",<29765872.1075851584037.JavaMail.evans@thyme>,2001-09-26 08:44:46-07:00,dana.davis@enron.com,"lesley.ayers@enron.com, bobbie.campbell@enron.com",RE: DASHes,NaN,1.0,text/plain; charset=us-ascii,7bit,NaN,"Davis, Dana </O=ENRON/OU=NA/CN=RECIPIENTS/CN=D...","Ayers, Lesley </O=ENRON/OU=NA/CN=RECIPIENTS/CN...",NaN,NaN,\DDAVIS (Non-Privileged)\Sent Items,DAVIS-D,DDAVIS (Non-Privileged).pst
20204,FYI:\n\n -----Original Message-----\nFrom: \tS...,<10205656.1075851584015.JavaMail.evans@thyme>,2001-09-26 09:21:08-07:00,dana.davis@enron.com,tammy.barta@enron.com,FW:,NaN,1.0,text/plain; charset=us-ascii,7bit,NaN,"Davis, Dana </O=ENRON/OU=NA/CN=RECIPIENTS/CN=D...","Barta, Tammy </O=ENRON/OU=NA/CN=RECIPIENTS/CN=...",NaN,NaN,\DDAVIS (Non-Privileged)\Sent Items,DAVIS-D,DDAVIS (Non-Privileged).pst
20205,\n\n -----Original Message-----\nFrom: \tWache...,<26331862.1075851583991.JavaMail.evans@thyme>,2001-09-26 11:34:36-07:00,dana.davis@enron.com,maria.lebeau@enron.com,"FW: Posting for Sr. Clerk, Job ID 0000110295",NaN,1.0,text/plain; charset=us-ascii,7bit,NaN,"Davis, Dana </O=ENRON/OU=NA/CN=RECIPIENTS/CN=D...","Lebeau, Maria </O=ENRON/OU=NA/CN=RECIPIENTS/CN...",NaN,NaN,\DDAVIS (Non-Privileged)\Sent Items,DAVIS-D,DDAVIS (Non-Privileged).pst
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21951,---------------------- Forwarded by Dana Davis...,<19409472.1075853965222.JavaMail.evans@thyme>,2001-02-20 03:18:00-08:00,dana.davis@enron.com,lola.willis@enron.com,(This is the 1st one that I took) Spiritual Gi...,NaN,1.0,text/plain; charset=ANSI_X3.4-1968,7bit,NaN,Dana Davis,Lola Willis,NaN,NaN,\Dana_Davis_Jun2001\Notes Folders\'sent mail,Davis-D,ddavis2.nsf
21952,\t\t\n\t\t\n\t\t\n\t\t\n\t\tPositive Living Ti...,<33200915.1075853963936.JavaMail.evans@thyme>,2001-02-20 04:38:00-08:00,dana.davis@enron.com,"bgacg@flash.net, abraham5@flash.net, kdavis207...",EMAZING Positive Living Tip of the Day - Your ...,NaN,1.0,text/plain; charset=us-ascii,7bit,NaN,Dana Davis,"bgacg@flash.net, abraham5@flash.net, kdavis207...",NaN,NaN,\Dana_Davis_Jun2001\Notes Folders\Sent,Davis-D,ddavis2.nsf
21953,\t\t\n\t\t\n\t\t\n\t\t\n\t\tPositive Living Ti...,<21463054.1075853965201.JavaMail.evans@thyme>,2001-02-20 04:38:00-08:00,dana.davis@enron.com,"bgacg@flash.net, abraham5@flash.net, kdavis207...",EMAZING Positive Living Tip of the Day - Your ...,NaN,1.0,text/plain; charset=us-ascii,7bit,NaN,Dana Davis,"bgacg@flash.net, abraham5@flash.net, kdavis207...",NaN,NaN,\Dana_Davis_Jun2001\Notes Folders\'sent mail,Davis-D,ddavis2.nsf
21954,Rufino should be here to assist you. \n\nThank...,<26492946.1075853963915.JavaMail.evans@thyme>,2001-02-20 05:09:00-08:00,dana.davis@enron.com,"brian.schwertner@enron.com, rosalyn.lum@enron....",Leaving Early,NaN,1.0,text/plain; charset=us-ascii,7bit,NaN,Dana Davis,"Brian Schwertner, Rosalyn Lum, Lisa Best, Lisa...",NaN,NaN,\Dana_Davis_Jun2001\Notes Folders\Sent,Davis-D,ddavis2.nsf


# END

In [271]:
employees[employees["Mail"].apply(lambda x: len(x)>1)]

,Name,Last,First,Mail
5,Phillip K Allen,Allen,Phillip,"[phillip.allen@enron.com, k..allen@enron.com]"
18,Sandra F Brawner,Brawner,Sandra,"[sandra.brawner@enron.com, f..brawner@enron.com]"
40,David W Delainey,Delainey,David,"[david.delainey@enron.com, w..delainey@enron.com]"
51,Daren J Farmer,Farmer,Daren,"[daren.farmer@enron.com, j..farmer@enron.com]"
55,John M Forney,Forney,John,"[john.forney@enron.com, m..forney@enron.com]"
60,Randall L Gay,Gay,Randall,"[randall.gay@enron.com, l..gay@enron.com]"
65,Darron C Giron,Giron,Darron,"[darron.giron@enron.com, c..giron@enron.com]"
70,Mark E Haedicke,Haedicke,Mark,"[mark.haedicke@enron.com, mark.e.haedicke@enro..."
85,Jeffrey T Hodge,Hodge,Jeffrey,"[jeffrey.hodge@enron.com, t..hodge@enron.com]"
87,Stanley Horton,Horton,Stanley,"[stanley.horton@enron.com, horton@enron.com]"
